In [1]:
import json
from tqdm import tqdm
import os, re
import numpy as np
import pandas as pd

In [2]:
def delete_tag(s):
    s = re.sub('\{IMG:.?.?.?\}', '', s)  # 图片
    s = re.sub(re.compile(r'[a-zA-Z]+://[^\s]+'), '', s)  # 网址
    s = re.sub(re.compile('<.*?>'), '', s)  # 网页标签
    s = re.sub(re.compile('&[a-zA-Z]+;?'), ' ', s)  # 网页标签
    s = re.sub(re.compile('[a-zA-Z0-9]*[./]+[a-zA-Z0-9./]+[a-zA-Z0-9./]*'), ' ', s)
    s = re.sub("\?{2,}", "", s)
    s = re.sub("\r", "", s)
    s = re.sub("\n", ",", s)
    s = re.sub("\t", ",", s)
    s = re.sub("（", ",", s)
    s = re.sub("）", ",", s)
    s = re.sub("\u3000", "", s)
    s = re.sub(" ", "", s)
    r4 = re.compile('\d{4}[-/]\d{2}[-/]\d{2}')  # 日期
    s = re.sub(r4, '某时', s)
    return s


In [3]:
def cut_sentences(content):
    # 结束符号，包含中文和英文的
    end_flag = ['。', ';', '；']

    content_len = len(content)
    sentences = []
    tmp_char = ''
    for idx, char in enumerate(content):
        # 拼接字符
        tmp_char += char

        # 判断是否已经到了最后一位
        if (idx + 1) == content_len:
            sentences.append(tmp_char)
            break

        # 判断此字符是否为结束符号
        if char in end_flag:
            # 再判断下一个字符是否为结束符号，如果不是结束符号，则切分句子
            next_idx = idx + 1
            if not content[next_idx] in end_flag:
                sentences.append(tmp_char)
                tmp_char = ''

    return sentences

In [4]:
def metl_data(df):
    z = df.groupby(['uid'])['content'].apply(lambda x: np.concatenate(list(x))).reset_index()
    i = pd.concat([pd.Series(row['uid'], row['content']) for _, row in z.iterrows()]).reset_index()
    i.columns = ['content', 'uid']
    return i


In [5]:
def get_data(train, test):

    train = train[~train.content_type.isnull()].drop_duplicates().reset_index(drop=True)
    train['content'] = train['content'].apply(lambda x: cut_sentences(x))
    train['content'] = list(map(lambda x, y: [i for i in x if y in i], train['content'], train['entity']))
    train_n = metl_data(train)
    train = train_n.merge(train[['uid', 'entity']], how='left')
    test['content'] = test['content'].apply(lambda x: cut_sentences(x))
    test = metl_data(test)
    train['content'] = train['content'].apply(lambda x: delete_tag(x))
    test['content'] = test['content'].apply(lambda x: delete_tag(x))

    train['content'] = list(map(lambda x, y: x[x.find(y) - 200:x.find(y) + 200], train['content'], train['entity']))
    return train, test



In [6]:
train = pd.read_csv('../ccks2020Data/event_entity_train_data_label.csv', sep='\t', header=None,
                        names=['uid', 'content', 'content_type', 'entity'])
test = pd.read_csv('../ccks2020Data/event_entity_dev_data.csv', header=None)
test = test[0].apply(lambda x: x.split('\t')).values.tolist()
test = pd.DataFrame(test, columns=['uid', 'content'])

In [7]:
train['len'] = train['content'].apply(lambda x: len(x))

In [9]:
train, test= get_data(train, test)

In [10]:
train

,content,uid,entity
0,皖通科技(002331)实控人杨世宁减持360万股比亚迪预计一季度业绩下滑开盘跌停,2001123,比亚迪
1,上海验配眼镜质量抽查:宝山申视宝铧等店不合格萨博销量下滑过半世爵已资不抵债,2001512,萨博
2,江苏金湖通报疫苗过期三大主因县疾控中心领导班子已全免职山东墨龙(002490)业绩“变脸”及...,2001958,山东墨龙
3,原标题：斐讯0元购陷阱买家数亿资金被套无法提现责任编辑：柯金定,2003028,斐讯
4,"2018年8月27日?-?就无法提现的问题,健康猫给出了以下处理方案:自签订协议起,每6个月...",2004193,健康猫
...,...,...,...
41413,其中包括，广弘控股、万润科技、中创物流、亿晶光电、航锦科技、清新环境、华铁科技、莱茵体育、卫...,3088237,航锦科技
41414,【红岸预警】科陆电子频繁变卖资产背后：财务危机深重公司持续亏损超12亿元资金或遭占用,3088377,科陆电子
41415,微信跑马游戏秘诀，根据精功集团此前公告，由于宏观去杠杆等原因，精功集团资金出现流动性困难，导...,3088387,精功集团
41416,中国重汽实控人将变更为山东省国资委9月29日晚间，中国重汽发布公告称，济南市国资委将其持有的...,3088825,中国重汽


In [11]:
test

,content,uid
0,"比如北京蓝天瑞德环保技术股份有限公司,,，经营业绩异常波动、涉及重大诉讼、公司银行账户被冻结...",2146341
1,03Ø风险中心由此线索深挖出福佑卡车数据造假、财务严重违规操作、多位核心高管知情并串谋、横跨...,2146779
2,2018年1月24日，公司发布了《海南航空控股股份有限公司重大资产重组停牌公告》(公告编号：...,2146933
3,三、被告广州市财京融资担保有限公司对被告鞠佳珍、谭厚兵的上述债务承担连带清偿责任。,2148028
4,4、2014年4月9日，詹庆辉向北京市海淀区人民法院申请司法轮候冻结合慧伟业持有本公司全部股...,2150490
...,...,...
1012,"#中经实时报#【百年人寿易主：绿城中国或接盘万达成第一大股东】12月17日晚间，绿城中国,,...",3141982
1013,"辽宁科隆精细化工股份有限公司,300405，sz,7月9日发布的一份公告显示，公司实际控制人...",3142853
1014,依据法院裁定，成都米米乐电子商务股份有限公司原实控人刘文太将手中所持成都米米乐电子商务股份有...,3143432
1015,此外，长沙水业拟认购公司非公开发行的全部亿股股票，转让后，长沙水业将持有%公司股份，成为惠博...,3145375


In [14]:
set(train['content'].apply(lambda x: len(x)))

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
